<p align="center">
    <img width=210 height=150 src="../reports/figures/cunef_bw.png">
</p>

<div>
<h1>2.0 Data Preprocessing </h1>
Fake News Classifier <br>
<strong>Ciencia de Datos para la Información No Estructurada</strong> <br>
<strong>Master Universidatio en Ciencia de Datos</strong>
</div>

<div style='text-align:right'>Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div>

---

En este notebook nos centramos en elaborar el modelo que utilizaremos para detectar las noticias verdaderas y falsas.

In [18]:
# Libraries
import pandas as pd
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [19]:
#nltk.download('wordnet')

### Lectura de datos

In [20]:
# Train data
X_train = pd.read_csv("../data/interim/train.csv")
Y_train = X_train['label']
X_train = X_train['title']

# Test data
X_test = pd.read_csv("../data/interim/test.csv")
Y_test = X_test['label']
X_test = X_test['title']

### Procesado de los datos
Realizaremos la limpieza de los datos, seguiremos el mismo procedimiento que en el análisis realizado. Crearemos una función que realizará la limpieza de los datos para cada fila del dataset, además incorporaremos la lematización con el objetivo de reducir la variabilidad de las palabras, verbos especialmente que permitirán manejar un menor vocabulario al modelo. 

In [21]:
# Translator to clean data 
translator = str.maketrans('', '', string.punctuation)

# Word lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

# Stopwords
stopwords = nltk.corpus.stopwords.words('english')

In [22]:
def title_cleaner(tlt):
    """
    Cleans the text passed. All text to lower, blanck spaces and punctuation.
    Also eliminates stopwords and lematize verbs
    :param tlt: text to clean
    :type tlt: string
    """
    # Lower letters and blank spaces out, we eliminate all type of punctuation
    # Finally we split the data (tokenize)
    tokens = re.sub(r"[^a-zA-Z0-9]", " ",tlt.strip().lower().translate(translator)).split()

    # Lemmatize and eliminate any kind of stopword that doesn't gives us any information
    final_words = [wordnet_lemmatizer.lemmatize(word, pos="v") for word in tokens if not word in stopwords]

    # Joining the resulted words cleaned
    return " ".join(final_words)

Aplicamos la función para limpiar los datos (tanto en train como en test)

In [23]:
X_train = X_train.map(lambda x : title_cleaner(x))
X_test = X_test.map(lambda x : title_cleaner(x))

Ahora codificamos la variable objetivo mediante LabelEncoding, es decir, 0 - FAKE y 1 - REAL

In [24]:
lb = preprocessing.LabelEncoder()
Y_train = lb.fit_transform(Y_train)
Y_test = lb.transform(Y_test)

print(lb.classes_)

['FAKE' 'REAL']


### Creación de variables del modelo
Ahora procedemos a crear las variables con los datos que serán utilizadas para el clasificador de noticias. Utilizaremos la técnica del Bag of Words y también la combinaremos con la técnica del TF-IDF. Crearemos muestras con solo el BoW y otra con el TF-IDF para poder utilizar cada una según el modelo y como consideremos oportuno.

In [25]:
# Model for creating variables
corpus = X_train
cv = CountVectorizer()
tft = TfidfTransformer()

# Creating the train result 
cv_matrix = cv.fit_transform(corpus)
tfd_result = tft.fit_transform(cv_matrix)

In [26]:
# Preparing train data
# BoW only
features_cv= cv.get_feature_names_out()
X_train_bow = pd.DataFrame(cv_matrix.toarray(), columns=features_cv)

# Tfd added
features_tft = tft.get_feature_names_out()
X_train_tfd = pd.DataFrame(tfd_result.toarray(), columns=features_tft)

In [27]:
# Preparing test data
# BoW only
data_matrix = cv.transform(X_test) 
X_test_bow = pd.DataFrame(data_matrix.toarray(), columns=features_cv)

# Tfd added
X_test_tfd = pd.DataFrame(tft.transform(data_matrix).toarray(), columns=features_tft)

### Guardado de los datos
Una vez procesados guardamos los datos

In [29]:
filenames = ["train_bow.csv", "train_tfd.csv", "test_bow.csv", "test_tfd.csv"]
samples = [X_train_bow, X_train_tfd, X_test_bow, X_test_tfd]
targets = [Y_train, Y_train, Y_test, Y_test]

for i in range(4):
    sample = samples[i]
    sample['label'] = targets[i]
    sample.to_csv("../data/processed/" + filenames[i])       

---

<div style='text-align:center'> Elaborado por Carlos Viñals Guitart (<i>carlos.vinals@cunef.edu</i>)</div>